In [29]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
import re
import pickle

In [30]:
file_path = 'health_insurance_fitness.csv'  # Replace with your file path
data = pd.read_csv(file_path)


In [31]:
def extract_numeric(value):
    if isinstance(value, str):
        value = re.sub(r'[^\d.]', '', value)  # Remove non-numeric characters
        return float(value) if value else 0.0
    return value


In [32]:
data['Coverage_Amount'] = data['Coverage_Amount'].apply(extract_numeric)
data['Monthly_Premium'] = data['Monthly_Premium'].apply(extract_numeric)
data['Annual_Premium'] = data['Annual_Premium'].apply(extract_numeric)
data['Claim_Settlement_Ratio'] = data['Claim_Settlement_Ratio'].apply(extract_numeric)

In [33]:
fitness_columns = ['Fitness_Score1', 'Fitness_Score2', 'Fitness_Score3', 'Fitness Score_4']
data['Composite_Fitness_Score'] = data[fitness_columns].mean(axis=1)

In [34]:
processed_data = data[['Brand_Name','Policy_Name','Cashless_Hospitals', 'Coverage_Amount', 'Monthly_Premium',
                       'Annual_Premium', 'Claim_Settlement_Ratio', 'Composite_Fitness_Score']]


In [35]:
scaler = MinMaxScaler()
normalized_features = scaler.fit_transform(processed_data[['Coverage_Amount', 'Monthly_Premium',
                                                           'Annual_Premium', 'Claim_Settlement_Ratio', 
                                                           'Composite_Fitness_Score']])

In [36]:
knn_model = NearestNeighbors(n_neighbors=12, algorithm='auto')
knn_model.fit(normalized_features)

NearestNeighbors(n_neighbors=12)

In [37]:
with open('knn_model.pkl', 'wb') as file:
    pickle.dump({
        'model': knn_model,
        'scaler': scaler,
        'data': processed_data
    }, file)
print("Model and preprocessing objects saved to knn_model.pkl")

Model and preprocessing objects saved to knn_model.pkl


In [38]:
with open('knn_model.pkl', 'rb') as file:
    saved_objects = pickle.load(file)

knn_model = saved_objects['model']
scaler = saved_objects['scaler']
processed_data = saved_objects['data']

print("Model and preprocessing objects loaded successfully.")


Model and preprocessing objects loaded successfully.


In [39]:
def recommend_policies(fitness_score, model, data, scaler):
    # Normalize the input fitness score with dummy data for other features
    input_features = [0, 0, 0, 0, fitness_score]
    normalized_input = scaler.transform([input_features])[0]
    
    # Find the nearest neighbors
    distances, indices = model.kneighbors([normalized_input])
    
    # Get the recommended policies
    recommendations = data.iloc[indices[0]]
    return recommendations[['Brand_Name','Policy_Name','Cashless_Hospitals', 'Coverage_Amount', 'Monthly_Premium', 
                             'Annual_Premium', 'Claim_Settlement_Ratio']]

In [40]:
fitness_score = 80
recommended_policies = recommend_policies(fitness_score, knn_model, processed_data, scaler)

c:\Users\nayak\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [41]:
print("Recommended Policies:")
print(recommended_policies)

Recommended Policies:
      Brand_Name                    Policy_Name  Cashless_Hospitals  \
138  Star Health    Arogya Sanjivini Individual                  16   
212  Star Health              Young Star Silver                  16   
77   Star Health  Mediclassic Individual Silver                  16   
78   Star Health    Arogya Sanjivini Individual                  16   
217  Star Health  Mediclassic Individual Silver                  16   
130  Star Health                Young Star Gold                  16   
139  Star Health               Smart Health Pro                  16   
129  Star Health              Young Star Silver                  16   
358  Star Health    Arogya Sanjivini Individual                  16   
221  Star Health    Arogya Sanjivini Individual                  16   
216  Star Health    Mediclassic Individual Gold                  16   
278  Star Health              Young Star Silver                  16   

     Coverage_Amount  Monthly_Premium  Annual_Premium 